# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [24]:
from dotenv import load_dotenv, find_dotenv
from utils import get_openai_api_key
_ = load_dotenv(override=True)
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [27]:
from autogen import ConversableAgent

## Creating the needed agents

In [28]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 01-18 16:30:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [29]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 01-18 16:30:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [30]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 01-18 16:30:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [31]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [32]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [33]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Joe and New York


customer_proxy_agent (to Onboarding Personal Information Agent):

Joe and New York

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name as Joe and your location as New York. Is there anything else you'd like to share with me?

--------------------------------------------------------------------------------


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Joe",
  "location": "New York"
}

--------------------------------------------------------------------------------


Provide feedback to Onboarding Topic preference Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Topic preference Agent):



--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Customer Engagement Agent):

Let's find something fun to read.
Context: 
{
  "name": "Joe",
  "location": "New York"
}
Joe is interested in knowing what topics the other person is interested in reading about.

--------------------------------------------------------------------------------
Customer Engagement Agent (to customer_proxy_agent):

Hey Joe! How about we start off with a fun fact? Did you know that New York City's Central Park is larger than the principality of Monaco? Pretty cool, right? Now, since you're interested in knowing what topics the other person is i

## Print out the summary

In [34]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Joe",
  "location": "New York"
}


Joe is interested in knowing what topics the other person is interested in reading about.


Joe is interested in knowing what topics the other person is interested in reading about. He can ask them about their favorite genres or authors to discover new books to read.




## Print out the cost

In [35]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0001675, 'gpt-3.5-turbo-0125': {'cost': 0.0001675, 'prompt_tokens': 197, 'completion_tokens': 46, 'total_tokens': 243}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001675, 'gpt-3.5-turbo-0125': {'cost': 0.0001675, 'prompt_tokens': 197, 'completion_tokens': 46, 'total_tokens': 243}}}


{'usage_including_cached_inference': {'total_cost': 5.8500000000000006e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8500000000000006e-05, 'prompt_tokens': 66, 'completion_tokens': 17, 'total_tokens': 83}}, 'usage_excluding_cached_inference': {'total_cost': 5.8500000000000006e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8500000000000006e-05, 'prompt_tokens': 66, 'completion_tokens': 17, 'total_tokens': 83}}}


{'usage_including_cached_inference': {'total_cost': 0.0003195, 'gpt-3.5-turbo-0125': {'cost': 0.0003195, 'prompt_tokens': 279, 'completion_tokens': 120, 'total_tokens': 399}}, 'usage_excluding_cached_inference': {'total_cost': 0.0003195, 'gpt-3.5-tur